In [5]:
import chromadb
from chromadb.config import Settings

persist_directory = r"C:/Users/Usuario/vectorialDatabaseCrud/chromadb"

# Updated client configuration
chroma_client = chromadb.PersistentClient(
    path=persist_directory
)

# Create collection for football matches
collection = chroma_client.create_collection(name="partidos_futbol")


In [ ]:
from sentence_transformers import SentenceTransformer

# Modelo de embeddings
modelo_embeddings = SentenceTransformer('all-MiniLM-L6-v2')

# Datos de partidos de fútbol
partidos = [
    {"id": "1", "texto": "Partido entre Barcelona y Real Madrid en el Camp Nou el 25 de diciembre de 2025.", "metadata": {"equipo_local": "Barcelona", "equipo_visitante": "Real Madrid", "fecha": "2025-12-25", "estadio": "Camp Nou"}},
    {"id": "2", "texto": "Partido entre Manchester United y Liverpool en Old Trafford el 10 de enero de 2025.", "metadata": {"equipo_local": "Manchester United", "equipo_visitante": "Liverpool", "fecha": "2025-01-10", "estadio": "Old Trafford"}},
    {"id": "3", "texto": "Partido entre Juventus y Inter de Milan en el Allianz Stadium el 5 de marzo de 2025.", "metadata": {"equipo_local": "Juventus", "equipo_visitante": "Inter de Milan", "fecha": "2025-03-05", "estadio": "Allianz Stadium"}},
]

# Generar embeddings para los textos
textos = [partido["texto"] for partido in partidos]
ids = [partido["id"] for partido in partidos]
metadatas = [partido["metadata"] for partido in partidos]
embeddings = modelo_embeddings.encode(textos).tolist()

# Insertar partidos con embeddings en la colección
collection.add(ids=ids, documents=textos, metadatas=metadatas, embeddings=embeddings)
print("Partidos con embeddings insertados.")


In [ ]:
# Generar embedding para la consulta
consulta = "Partido entre Barcelona y Real Madrid en el Camp Nou"
embedding_consulta = modelo_embeddings.encode([consulta]).tolist()

# Buscar los documentos más similares
resultados_similares = collection.query(query_embeddings=embedding_consulta, n_results=2)
print("Partidos similares:", resultados_similares)


In [ ]:
# Actualizar un partido y su embedding
nuevo_texto = "Partido entre Barcelona y Real Madrid en el Camp Nou el 25 de diciembre de 2025, con 70,000 espectadores."
nuevo_embedding = modelo_embeddings.encode([nuevo_texto]).tolist()
collection.delete(ids=["1"])  # Eliminar partido antiguo
collection.add(ids=["1"], documents=[nuevo_texto], metadatas=[{"equipo_local": "Barcelona", "equipo_visitante": "Real Madrid", "fecha": "2025-12-25", "estadio": "Camp Nou"}], embeddings=nuevo_embedding)
print("Partido actualizado con nuevo embedding.")


In [ ]:
# Eliminar un partido
collection.delete(ids=["2"])
print("Partido eliminado.")


In [ ]:
client.persist()

In [ ]:
from IPython.display import clear_output
clear_output(wait=True)

In [ ]:
import chromadb

# Inspeccionar las instancias activas
print(chromadb.api.ClientCreator._identifier_to_system)


In [ ]:
from IPython.display import display, Javascript
display(Javascript("Jupyter.notebook.kernel.restart()"))


In [ ]:
import os

# Verifica si la carpeta chromadb existe
path = "./chromadb"
if os.path.exists(path):
    print(f"La carpeta existe en: {os.path.abspath(path)}")
else:
    print("La carpeta 'chromadb' no fue creada.")
